In [1]:
from __future__ import division

import re
import sys
import os

import random
import math

import numpy as np
import pandas as pd
import scipy

import gensim, logging

import pymorphy2
from nltk import word_tokenize
from nltk.tokenize import TreebankWordTokenizer
from stop_words import get_stop_words

from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

morph = pymorphy2.MorphAnalyzer()
tokenizer = TreebankWordTokenizer()

RUS_LETTERS = u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Сначала нужно провести предобработку текстов: лемматизировать, удалить стоп-слова, возможно, разметить части речи (но последнее необязательно).

In [2]:
anekdots = [f for f in os.listdir('./weka/anekdots/') if f.endswith('.txt')]

In [3]:
izvest = [f for f in os.listdir('./weka/izvest/') if f.endswith('.txt')]

In [4]:
teh_mol = [f for f in os.listdir('./weka/teh_mol/') if f.endswith('.txt')]

In [5]:
len(anekdots)

125

In [6]:
m = 'ruscorpora_1_300_10.bin'
if m.endswith('.vec'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.Word2Vec.load(m)

2017-03-17 22:57:56,480 : INFO : loading projection weights from ruscorpora_1_300_10.bin
2017-03-17 22:58:01,913 : INFO : loaded (184973, 300) matrix from ruscorpora_1_300_10.bin


In [7]:
arr_corpora = []

for a in anekdots:
    with open('./weka/anekdots/' + a, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = 'anekdots'
        dicti['text'] = text
        arr_corpora.append(dicti)

for i in izvest:
    with open('./weka/izvest/' + i, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = 'izvest'
        dicti['text'] = text
        arr_corpora.append(dicti)
        
for t in teh_mol:
    with open('./weka/teh_mol/' + t, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = 'teh_mol'
        dicti['text'] = text
        arr_corpora.append(dicti)
        
        
random.shuffle(arr_corpora)  # перемешиваем данные
    
df = pd.DataFrame(arr_corpora)  # создаем датасет

In [8]:
data_train, data_test, class_train, class_test = train_test_split(df['text'], df['class'], test_size=0.2)

In [9]:
df.head()

,class,text
0,anekdots,Шлюхи\n***\nВ публичный дом приходит клиент и ...
1,anekdots,"Ходжа Насреддин\n***\nНа вопрос соседа, что ну..."
2,izvest,Лесков Сергей. Электронный бум -- не фикция. Н...
3,anekdots,Хохлы\n***\nКупил новый украинец себе шестисот...
4,teh_mol,Г. Малиничев. Десять заповедей инженера\nНедав...


In [10]:
def do_smth_with_model(steps):
    print('\nModel train')
    pipeline = Pipeline(steps=steps)

    cv_results = cross_val_score(pipeline,
                                 data_train,
                                 class_train,
                                 cv=10,
                                 scoring='accuracy',
                                )
    print(cv_results.mean(), cv_results.std())

    pipeline.fit(data_train, class_train)
    class_predicted = pipeline.predict(data_test)
    print(class_predicted)

    print(classification_report(class_test, class_predicted ))

    return pipeline, class_predicted

In [11]:
transit = {'ADJF':'ADJ', 'INFN':'VERB', 'ADVB':'ADV'}
robj = re.compile('|'.join(transit.keys()))

def cleanization(text):
    for line in text:
        # 1. Все буквы в нижний регистр
        text_text = text.lower()

        # 2. Удаление всех небукв
        letters_only = ''
        for _c in text_text:
            if _c in RUS_LETTERS:
                letters_only += _c
            else:
                letters_only += ' '

        # 3. Заменяем множественные пробелы
        while '  ' in letters_only:
            letters_only = letters_only.replace('  ', ' ')

        # 4. Токенизация
        word_list = tokenizer.tokenize(letters_only)

        # 5. Лемматизация
        clean_word_list = [morph.parse(word)[0].normal_form for word in word_list]  # лемматизация
    
        # 6. Удаление стоп-слов + добавление тегов - части речи
        meaningful_words = [str(word) + '_' + robj.sub(lambda m: transit[m.group(0)], str(morph.parse(word)[0].tag.POS)) for word in clean_word_list if word not in get_stop_words('ru')] # стоп-слова
        return ' '.join(meaningful_words)   # meaningful_words

In [12]:
from __future__ import division
def mean(a):
    return sum(a) / len(a)

In [13]:
def numbers(text):
    """Сколько чисел."""
    arr = []
    clean_text = cleanization(text)
    # для каждого слова в тексте выводим его вектор
    for word in clean_text.split():
    # есть ли слово в модели? Может быть, и нет
        if word in model:
            arr.append(model[word])
    # print(len(list(map(mean, zip(*arr)))))
    return list(map(mean, zip(*arr)))



class FunctionFeaturizer(TransformerMixin):
    """ Для создания своего вектора я использовала несколько фич: длину текста, количество заглавных букв
     (чем больше, тем обычно выше вероятность, что это спам), количество ! (в спам-сообщениях встречаются часто),
     количество чисел, сколько слов из словаря спам-слов (50 самых частых слов в коллекции спам-сообщений)"""
    def __init__(self, featurizers):
        self.featurizers = featurizers

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        fvs = []
        for datum in X:
            fv = numbers(datum)
            fvs.append(fv)
        return np.array(fvs)

In [14]:
spam_featurizer = FunctionFeaturizer(numbers)  # создание своего векторизатора

In [15]:
# numbers('Для создания своего вектора я использовала несколько фич: длину текста, количество заглавных букв')

In [75]:
# Свой векторизатор
print('\nCustom Transformer')
pipeline, label_predicted = do_smth_with_model(steps=[('custom', spam_featurizer),
                                                      ('classifier', LogisticRegression())])


Custom Transformer

Model train
0.860307749351 0.0796299815339
['teh_mol' 'izvest' 'anekdots' 'anekdots' 'izvest' 'izvest' 'anekdots'
 'anekdots' 'anekdots' 'anekdots' 'teh_mol' 'izvest' 'anekdots' 'izvest'
 'anekdots' 'teh_mol' 'izvest' 'teh_mol' 'izvest' 'teh_mol' 'izvest'
 'anekdots' 'anekdots' 'izvest' 'anekdots' 'izvest' 'izvest' 'anekdots'
 'teh_mol' 'teh_mol' 'izvest' 'anekdots' 'teh_mol' 'teh_mol' 'izvest'
 'teh_mol' 'izvest' 'teh_mol' 'teh_mol' 'teh_mol' 'anekdots' 'izvest'
 'teh_mol' 'anekdots' 'izvest' 'anekdots' 'izvest' 'anekdots' 'anekdots'
 'anekdots' 'izvest' 'izvest' 'izvest' 'izvest' 'teh_mol' 'izvest'
 'anekdots' 'anekdots' 'anekdots' 'izvest' 'izvest' 'teh_mol' 'izvest'
 'teh_mol' 'izvest' 'anekdots' 'teh_mol' 'teh_mol' 'anekdots' 'teh_mol'
 'anekdots' 'anekdots' 'izvest' 'izvest' 'izvest']
             precision    recall  f1-score   support

   anekdots       0.96      1.00      0.98        25
     izvest       0.79      0.88      0.84        26
    teh_mol      

In [ ]:
# Свой векторизатор
print('\nCustom Transformer')
pipeline, label_predicted = do_smth_with_model(steps=[('custom', spam_featurizer),
                                                      ('classifier', LogisticRegression(penalty="l2", solver="lbfgs", multi_class="multinomial", max_iter=300, n_jobs=4))])


Custom Transformer

Model train
